In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from torch.utils.data import DataLoader, TensorDataset, Subset
import sklearn.metrics as sk

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
data = np.load("/content/drive/MyDrive/Unilever Datasets, Sliding Window 13/HEALTHY SNACKING_dataset_processed (1).npz")

In [4]:

# Extract the arrays
X = data['X']  # Shape: (timesteps, input_sequence_length, nodes, features)
Y = data['Y']  # Shape: (timesteps, output_sequence_length, nodes, features)

# Drop all features except the first one for both X and Y
X = X[:, :, :, 0]  # Keep only the first feature
Y = Y[:, :, :, 0]  # Keep only the first feature

In [5]:
# Reshape Y to match the output format, if necessary
# Y is expected to have shape (timestep/index, prediction window, nodes, 1)
# If Y's shape is already correct, skip this step
X = X.squeeze(-1)
Y = Y.squeeze(-1)  # Remove the last dimension

ValueError: cannot select an axis to squeeze out which has size not equal to one

In [6]:

# Verify that the input and output sequence lengths are 13
assert X.shape[1] == 13, f"Expected input sequence length of 13, got {X.shape[1]}"
assert Y.shape[1] == 13, f"Expected output sequence length of 13, got {Y.shape[1]}"

# Split the data into train, validation, and test sets (70%, 15%, 15%)
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.15)
test_size = len(X) - train_size - val_size

X_train, X_val, X_test = X[:train_size], X[train_size:train_size + val_size], X[train_size + val_size:]
y_train, y_val, y_test = Y[:train_size], Y[train_size:train_size + val_size], Y[train_size + val_size:]

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Print shapes to verify
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)



X_train shape: torch.Size([116, 13, 1082])
y_train shape: torch.Size([116, 13, 1082])
X_val shape: torch.Size([25, 13, 1082])
y_val shape: torch.Size([25, 13, 1082])
X_test shape: torch.Size([26, 13, 1082])
y_test shape: torch.Size([26, 13, 1082])


In [7]:
# Define the GRU model
class AirModel(nn.Module):
    def __init__(self, input_sz, hidden_sz, output_sz, n_steps, dropout_prob=0.1):
        super().__init__()
        self.gru = nn.GRU(
            input_size=input_sz,
            hidden_size=hidden_sz,
            num_layers=2,
            batch_first=True
        )
        self.dropout = nn.Dropout(p=dropout_prob)
        self.linear = nn.Linear(hidden_sz, output_sz * n_steps)
        self.output_sz = output_sz
        self.n_steps = n_steps

    def forward(self, x):
        x, _ = self.gru(x)
        x = self.dropout(x[:, -1, :])  # Use the output from the last time step
        x = self.linear(x)
        x = torch.relu(x)
        x = x.view(-1, self.n_steps, self.output_sz)  # Reshape to match the output format
        return x

# Model parameters
input_size = X_train.shape[2]
output_size = y_train.shape[2]
model = AirModel(
    input_sz=input_size,
    hidden_sz=50,
    output_sz=output_size,
    n_steps=13,  # Updated to match the forecasting window
    dropout_prob=0.1  # Added dropout for regularization
)

# Optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Reduced learning rate for stability
loss_fn = nn.MSELoss()

# Prepare the data loader
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=16, shuffle=False)

# Lists to store metrics
train_metrics = {'RMSE': [], 'MAE': [], 'R2': [], 'MSE': [], 'MAPE': [], 'WMAPE': []}
val_metrics = {'RMSE': [], 'MAE': [], 'R2': [], 'MSE': [], 'MAPE': [], 'WMAPE': []}
test_metrics = {'RMSE': [], 'MAE': [], 'R2': [], 'MSE': [], 'MAPE': [], 'WMAPE': []}

# Function to calculate WMAPE
def wmape(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))


# Function to calculate MAPE
def mape(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    # Create a mask for non-zero actuals
    mask = y_true != 0
    # Ensure there are non-zero actuals to prevent division by zero in the mean
    if np.sum(mask) == 0:
        return np.nan  # Or handle appropriately
    # Compute MAPE only on non-zero actuals
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100


# Training loop
n_epochs = 500  # Reduced epochs for faster training during testing
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation every epoch
    model.eval()
    with torch.no_grad():
        # Helper function to calculate metrics
        def calculate_metrics(y_true, y_pred, n):
            y_true = y_true[:, :n, :].cpu().numpy()
            y_pred = y_pred[:, :n, :].cpu().numpy()

            mse = mean_squared_error(y_true.flatten(), y_pred.flatten())
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_true.flatten(), y_pred.flatten())
            r2 = r2_score(y_true.flatten(), y_pred.flatten())
            mape_value = mape(y_true, y_pred)
            wmape_value = wmape(y_true, y_pred) * 100  # Expressed as percentage
            return mse, rmse, mae, r2, mape_value, wmape_value

        # Training metrics
        y_pred_train = model(X_train)
        mse_train, rmse_train, mae_train, r2_train, mape_train, wmape_train = calculate_metrics(y_train, y_pred_train, 13)
        train_metrics['MSE'].append(mse_train)
        train_metrics['RMSE'].append(rmse_train)
        train_metrics['MAE'].append(mae_train)
        train_metrics['R2'].append(r2_train)
        train_metrics['MAPE'].append(mape_train)
        train_metrics['WMAPE'].append(wmape_train)

        # Validation metrics
        y_pred_val = model(X_val)
        mse_val, rmse_val, mae_val, r2_val, mape_val, wmape_val = calculate_metrics(y_val, y_pred_val, 13)
        val_metrics['MSE'].append(mse_val)
        val_metrics['RMSE'].append(rmse_val)
        val_metrics['MAE'].append(mae_val)
        val_metrics['R2'].append(r2_val)
        val_metrics['MAPE'].append(mape_val)
        val_metrics['WMAPE'].append(wmape_val)

        # Testing metrics
        y_pred_test = model(X_test)
        mse_test, rmse_test, mae_test, r2_test, mape_test, wmape_test = calculate_metrics(y_test, y_pred_test, 13)
        test_metrics['MSE'].append(mse_test)
        test_metrics['RMSE'].append(rmse_test)
        test_metrics['MAE'].append(mae_test)
        test_metrics['R2'].append(r2_test)
        test_metrics['MAPE'].append(mape_test)
        test_metrics['WMAPE'].append(wmape_test)

    # Print metrics every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}")
        print(f"Train - RMSE: {rmse_train:.4f}, MAE: {mae_train:.4f}, R²: {r2_train:.4f}, MAPE: {mape_train:.2f}%, WMAPE: {wmape_train:.2f}%")
        print(f"Val   - RMSE: {rmse_val:.4f}, MAE: {mae_val:.4f}, R²: {r2_val:.4f}, MAPE: {mape_val:.2f}%, WMAPE: {wmape_val:.2f}%")
        print(f"Test  - RMSE: {rmse_test:.4f}, MAE: {mae_test:.4f}, R²: {r2_test:.4f}, MAPE: {mape_test:.2f}%, WMAPE: {wmape_test:.2f}%")
        print("-" * 80)

Epoch 10/500
Train - RMSE: 1282.7100, MAE: 206.7632, R²: -0.0256, MAPE: 96.76%, WMAPE: 100.14%
Val   - RMSE: 1404.7397, MAE: 196.9567, R²: -0.0193, MAPE: 97.62%, WMAPE: 100.16%
Test  - RMSE: 870.9279, MAE: 92.2407, R²: -0.0103, MAPE: 97.89%, WMAPE: 100.80%
--------------------------------------------------------------------------------
Epoch 20/500
Train - RMSE: 1282.0996, MAE: 207.0038, R²: -0.0247, MAPE: 95.72%, WMAPE: 100.25%
Val   - RMSE: 1404.2490, MAE: 197.2177, R²: -0.0186, MAPE: 96.43%, WMAPE: 100.30%
Test  - RMSE: 870.5641, MAE: 92.9298, R²: -0.0095, MAPE: 96.61%, WMAPE: 101.55%
--------------------------------------------------------------------------------
Epoch 30/500
Train - RMSE: 1281.5258, MAE: 207.2101, R²: -0.0238, MAPE: 95.11%, WMAPE: 100.35%
Val   - RMSE: 1403.7872, MAE: 197.4409, R²: -0.0179, MAPE: 95.65%, WMAPE: 100.41%
Test  - RMSE: 870.2241, MAE: 93.5490, R²: -0.0087, MAPE: 95.76%, WMAPE: 102.23%
-------------------------------------------------------------------

In [8]:
y_true_flat = y_train.cpu().numpy().flatten()
num_zeros = np.sum(y_true_flat == 0)
print(f"Number of zero values in y_true: {num_zeros}")

Number of zero values in y_true: 1413067


In [9]:
total_values = y_true_flat.size  # Assuming y_true_flat is a flattened version of y_true
percentage_zeros = (num_zeros / total_values) * 100
print(f"Percentage of zero values in y_true: {percentage_zeros:.2f}%")

Percentage of zero values in y_true: 86.60%


In [10]:
# Calculate metrics for t=1 to t=5 and t=1 to t=13
def calculate_final_metrics(y_true, y_pred, n_steps_list):
    results = {}
    for n in n_steps_list:
        mse, rmse, mae, r2, mape_value, wmape_value = calculate_metrics(y_true, y_pred, n)
        results[f'n={n}'] = {
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2,
            'MAPE': mape_value,
            'WMAPE': wmape_value
        }
    return results

# Evaluate on train set
model.eval()
with torch.no_grad():
    y_pred_train = model(X_train)
    train_results = calculate_final_metrics(y_train, y_pred_train, [5, 13])

    y_pred_val = model(X_val)
    val_results = calculate_final_metrics(y_val, y_pred_val, [5, 13])

    y_pred_test = model(X_test)
    test_results = calculate_final_metrics(y_test, y_pred_test, [5, 13])

# Print final metrics
def print_results(results, dataset_name):
    print(f"\n{dataset_name} Metrics:")
    for n, metrics in results.items():
        print(f"\nFrom t=1 to t={n.split('=')[1]}:")
        print(f"MSE: {metrics['MSE']:.4f}")
        print(f"RMSE: {metrics['RMSE']:.4f}")
        print(f"MAE: {metrics['MAE']:.4f}")
        print(f"R²: {metrics['R2']:.4f}")
        print(f"MAPE: {metrics['MAPE']:.2f}%")
        print(f"WMAPE: {metrics['WMAPE']:.2f}%")

print_results(train_results, "Train")
print_results(val_results, "Validation")
print_results(test_results, "Test")


Train Metrics:

From t=1 to t=5:
MSE: 1581413.3750
RMSE: 1257.5426
MAE: 210.9650
R²: 0.0126
MAPE: 87.43%
WMAPE: 102.24%

From t=1 to t=13:
MSE: 1583369.0000
RMSE: 1258.3199
MAE: 211.0489
R²: 0.0130
MAPE: 87.42%
WMAPE: 102.21%

Validation Metrics:

From t=1 to t=5:
MSE: 1866532.1250
RMSE: 1366.2108
MAE: 203.6246
R²: 0.0085
MAPE: 88.48%
WMAPE: 102.68%

From t=1 to t=13:
MSE: 1919276.1250
RMSE: 1385.3794
MAE: 202.4178
R²: 0.0086
MAPE: 89.49%
WMAPE: 102.94%

Test Metrics:

From t=1 to t=5:
MSE: 794632.8750
RMSE: 891.4218
MAE: 110.9237
R²: 0.0168
MAPE: 90.45%
WMAPE: 122.90%

From t=1 to t=13:
MSE: 736112.3125
RMSE: 857.9698
MAE: 112.1796
R²: 0.0195
MAPE: 92.57%
WMAPE: 122.58%
